In [36]:
using Pkg
using Distributions
using Plots
using JLD2

In [1]:
L = 200;
u = 0.0001;
k1 = 1;
k2 = 0.04;
eps = 0.01;  
alpha = 0.1;
F0 = 1;
m = 1;
h = 0.1;
eps2 = 0.007;    

roll (generic function with 1 method)

In [79]:
x = 0.1 * rand(Uniform(-1,1), L);
v = zeros(L);
t = 0;
# x,v,t,eventsSize,eventsBlocks,eventsBlocks2,xdata,vdata,ends,tenstionData = run(x,v,t,1_000_000,L,alpha)
# Ts = tenstionData[ends][np.argwhere(eventsBlocks2[:] >= 500)[:,0]]

In [80]:
function roll(arr, step)
    return vcat(arr[end-step+1:end], arr[1:end-step])
end

function acc(x,v,t,l,alpha)
    k2= 1/(l-1)
    xIntoLeft = roll(x,size(x)[1]-1)
    xIntoRight = roll(x,1)

    return (-x*(2*k1 + k2) .+ (xIntoRight + xIntoLeft)*k1 .+ k2*u*t .- f(x,v,t,l,alpha) )/m
end;

function f(x,v,t,l,alpha)
    k2= 1/(l-1)
    
    xIntoLeft = roll(x,size(x)[1]-1)
    xIntoRight = roll(x,1)

    return ifelse.(v .==0 , ifelse.(abs.(-x*(2*k1 + k2) .+ (xIntoRight + xIntoLeft)*k1 .+ k2*u*t) .< F0,-x*(2*k1 + k2) .+ (xIntoRight + xIntoLeft)*k1 .+ k2*u*t ,F0*(1-eps)),F0*(1-eps) ./ (1 .+ alpha*abs.(v)))

end;

function tension(x,v,t,l,alpha)
    return m*acc(x,v,t,l,alpha) .+ f(x,v,t,l,alpha)
end 

function doStep(x,v,t,l,alpha)
    k2= 1/(l-1)


    if all(v .== 0)
        t += (F0-maximum(tenstion(x,v,t,l,alpha)))/(k2*u)

        k_1 = v
        l_1 = acc(x,v,t,l,alpha)

        k_2 = v .+ l_1*h/2
        l_2 = acc(x .+ k_1*h/2,v .+ l_1*h/2,t + h/2,l,alpha)

        k_3 = v .+ l_2*h/2
        l_3 = acc(x .+ k_2*h/2,v .+ l_2*h/2,t + h/2,l,alpha)

        k_4 = v .+ l_3/2
        l_4 = acc(x .+ k_3*h,v .+ l_3*h, t + h,l,alpha)

        x = x +  (k_1 + 2*k_2 + 2*k_3 + k_4)*h/6
        v = v +  (l_1 .+ 2*l_2 .+ 2*l_3 + l_4)*h/6

        v = ifelse.(v .< 0 , 0 , v)

    else
        k_1 = v
        l_1 = acc(x,v,t,l,alpha)

        k_2 = v .+ l_1*h/2
        l_2 = acc(x .+ k_1*h/2,v .+ l_1*h/2,t + h/2,l,alpha)

        k_3 = v .+ l_2*h/2
        l_3 = acc(x .+ k_2*h/2,v .+ l_2*h/2,t + h/2,l,alpha)

        k_4 = v .+ l_3/2
        l_4 = acc(x .+ k_3*h,v .+ l_3*h, t + h,l,alpha)

        x = x +  (k_1 + 2*k_2 + 2*k_3 + k_4)*h/6
        v = v +  (l_1 + 2*l_2 + 2*l_3 + l_4)*h/6

        v = ifelse.(v .< 0 , 0 , v)

    end

    return x, v,t+h

end

In [138]:
xdata = zeros((10,10))
vdata = zeros((10,10))

10×10 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [140]:
for i in range(1,10)
    println(i)
end

1
2
3
4
5
6
7
8
9
10


In [189]:
l = L 
steps = 10 


xdata = zeros((steps,l))
vdata = zeros((steps,l))
tenstionData = zeros((steps,l))
for i in range(1,steps)
    x,v,t = doStep(x,v,t,l,alpha)
    xdata[i,:] = x
    vdata[i,:] = v
    tenstionData[i,:] = tenstion(x,v,t,l,alpha)

end 

slipping = any(vdata .>0 , dims = 2)

slippingR = roll(slipping,+1)
slippingL = roll(slipping,size(slipping)[1]-1)

slippingR[1] = false
slippingL[size(slipping)[1]] = false

ends  = slipping .⊻ (slipping[:,1] .& slippingL)
starts  = slipping .⊻ (slipping[:,1] .& slippingR)

totalX = sum(xdata,dims = 2)


10×1 Matrix{Float64}:
 -0.7653150611932252
 -0.7647366681363874
 -0.7642190188436765
 -0.7637723172192169
 -0.7634053051577223
 -0.763125091748077
 -0.7629370159000058
 -0.7628445450391385
 -0.7628492117819378
 -0.762949855558217

In [192]:
ends

10×1 BitMatrix:
 0
 0
 0
 0
 0
 0
 0
 0
 1
 0

In [202]:
findall(x -> x == 0, ends[:,1])

9-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
 10

In [205]:
v[findall(x -> x == 1, ends[:,1])]

1-element Vector{Real}:
 0.0

In [207]:
totalX

10×1 Matrix{Float64}:
 -0.7653150611932252
 -0.7647366681363874
 -0.7642190188436765
 -0.7637723172192169
 -0.7634053051577223
 -0.763125091748077
 -0.7629370159000058
 -0.7628445450391385
 -0.7628492117819378
 -0.762949855558217

In [206]:
totalX[findall(x -> x == 1, ends[:,1])]

1-element Vector{Float64}:
 -0.7628492117819378

In [221]:
[findall(x -> x == 1, ends[:,1])]

1-element Vector{Vector{Int64}}:
 [9]

In [237]:
sum(xdata[findall(x -> x == 1, ends[:,1]),:]  .!= xdata[findall(x -> x == 1, starts[:,1]),:],dims=2)

1×1 Matrix{Int64}:
 1

1×200 Matrix{Float64}:
 0.301747  0.037456  0.0328477  …  0.0974172  -0.0167453  -0.0481326

In [187]:
function doRun(x,v,t,steps,l,alpha)
    xdata = zeros((steps,l))
    vdata = zeros((steps,l))
    tenstionData = zeros((steps,l))
    for i in range(1,steps)
        x,v,t = doStep(x,v,t,l,alpha)
        xdata[i,:] = x
        vdata[i,:] = v
        tenstionData[i,:] = tenstion(x,v,t,l,alpha)

    end 

    slipping = any(vdata .>0 , dims = 2)

    slippingR = roll(slipping,+1)
    slippingL = roll(slipping,size(slipping)[1]-1)

    slippingR[1] = false
    slippingL[size(slipping)[1]] = false

    ends  = slipping .⊻ (slipping[:,1] .& slippingL)
    ends = findall(x -> x == 1, ends[:,1])

    starts  = slipping .⊻ (slipping[:,1] .& slippingR)
    starts = findall(x -> x == 1, starts[:,1])



    totalX = sum(xdata,dims = 2)

    eventsSize = totalX[ends] - totalX[starts]

    eventsBlocks = sum(xdata[findall(x -> x == 1, ends[:,1]),:]  .!= xdata[findall(x -> x == 1, starts[:,1]),:],dims=2)

    arr1 = vdata .>0;
    arr2 = arr1 .> circshift(arr1,(0,1));
    arr3 = sum(arr2,dims=2)
    arr4 = cumsum(arr3,dims=1)   ###CHECK

    arr5 = arr4[starts]
    arr6 = arr4[ends]

    eventsBlocks2 = arr6-arr5


    return x,v,t,eventsSize,eventsBlocks,eventsBlocks2,xdata,vdata,ends,tenstionData


    end



doRun (generic function with 1 method)

In [264]:
doRun(x,v,t,1_000_000, L,alpha)

LoadError: InterruptException:

In [257]:

x,v,t,eventsSize,eventsBlocks,eventsBlocks2,xdata,vdata,ends,tenstionData = doRun(x,v,t,100, L,alpha)

LoadError: BoundsError: attempt to access Tuple{Vector{Float64}, Vector{Float64}, Float64, Matrix{Float64}} at index [5]

In [260]:
l = L 
steps = 1000

xdata = zeros((steps,l))
vdata = zeros((steps,l))
tenstionData = zeros((steps,l))
for i in range(1,steps)
    x,v,t = doStep(x,v,t,l,alpha)
    xdata[i,:] = x
    vdata[i,:] = v
    tenstionData[i,:] = tenstion(x,v,t,l,alpha)

end 

slipping = any(vdata .>0 , dims = 2)

slippingR = roll(slipping,+1)
slippingL = roll(slipping,size(slipping)[1]-1)

slippingR[1] = false
slippingL[size(slipping)[1]] = false

ends  = slipping .⊻ (slipping[:,1] .& slippingL)
ends = findall(x -> x == 1, ends[:,1])

starts  = slipping .⊻ (slipping[:,1] .& slippingR)
starts = findall(x -> x == 1, starts[:,1])



totalX = sum(xdata,dims = 2)

eventsSize = totalX[ends] - totalX[starts]

eventsBlocks = sum(xdata[findall(x -> x == 1, ends[:,1]),:]  .!= xdata[findall(x -> x == 1, starts[:,1]),:],dims=2)

arr1 = vdata .>0;
arr2 = arr1 .> circshift(arr1,(0,1));
arr3 = sum(arr2,dims=2)
arr4 = cumsum(arr3,dims=1)   ###CHECK

arr5 = arr4[starts]
arr6 = arr4[ends]

eventsBlocks2 = arr6-arr5

arr6 - arr5 

44-element Vector{Int64}:
 18
 21
 21
 21
 21
 21
 21
 21
 21
 21
 21
 21
 21
  ⋮
 21
 21
 21
 21
 21
 21
 21
 21
 21
 21
 21
 13